In [27]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

# Random Graph Identification

Determining which algorithm created certain graphs by their properties.

In [37]:
    P1_Graphs = pickle.load(open('A4_graphs','rb'))
    P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)


In [41]:

#look for high degrees in PA
#look for high clustering coef in sW_L
G=P1_Graphs[4]
G.nodes()

AttributeError: 'Graph' object has no attribute '_node'

In [18]:

def graph_identification():
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
        if len(histogram)>10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    return methods

In [6]:
graph_identification()

AttributeError: 'Graph' object has no attribute '_adj'

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [43]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))


FileNotFoundError: [Errno 2] No such file or directory: 'email_prediction.txt'

In [189]:
df=pd.DataFrame(index=G.nodes())
#df['Department']=nx.get_node_attributes(G,'Department')
df['ManagementSalary']=pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df['Department']=pd.Series(nx.get_node_attributes(G, 'Department'))
df['clustering']=pd.Series(nx.clustering(G))
df['degree']=pd.Series(nx.degree(G))
df.head()

,ManagementSalary,Department,clustering,degree
0,0.0,1,0.276423,44
1,NaN,1,0.265306,52
2,NaN,21,0.297803,95
3,1.0,21,0.384910,71
4,1.0,21,0.318691,96


In [190]:
train=df[~pd.isnull(df['ManagementSalary'])]

test=df[pd.isnull(df['ManagementSalary'])]

y_train=train['ManagementSalary']
X_train=train.iloc[:,[1,2,3]]

y_test=test['ManagementSalary']
X_test=test.iloc[:,[1,2,3]]




### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [191]:
def salary_predictions():
    
    from sklearn.linear_model import LogisticRegression
    lrmodel=LogisticRegression()
    lrmodel.fit(X_train,y_train)

    final=pd.Series([i for i in zip(*lrmodel.predict_proba(X_test))][1],index=X_test.index)
    
    return final

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)
future_connections.shape


(488446, 1)

In [ ]:
'''df=pd.DataFrame(index=[(x[0],x[1]) for x in list(nx.preferential_attachment(G))])
future_connections.sort_index(inplace=True)
df.sort_index(inplace=True)


df['connection']=future_connections['Future Connection']

df['Pref']=[x[2] for x in PA]

df['Common Neighbors'] = df.index.map(lambda node: len(list(nx.common_neighbors(G, node[0], node[1]))))

df['jacc_coef']=[x[2] for x in list(nx.jaccard_coefficient(G))]

df['resource_alloc']=[x[2] for x in list(nx.resource_allocation_index(G))]

df['adamic_adar']=[x[2] for x in list(nx.adamic_adar_index(G))]


##commnity measures
for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']

df['cn_sound']=[x[2] for x in list(nx.cn_soundarajan_hopcroft(G))]

df['ra_sound']=[x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G))]
'''

In [ ]:
'''testindex=future_connections[pd.isnull(future_connections['Future Connection'])].index
trainindex=future_connections[~pd.isnull(future_connections['Future Connection'])].index
features=['Pref', 'Common Neighbors', 'jacc_coef', 'resource_alloc',
       'adamic_adar', 'cn_sound', 'ra_sound']



y_train=df.loc[trainindex,'connection']
X_train=df.loc[trainindex,features]

y_test=df.loc[testindex,'connection']
X_test=df.loc[testindex,features]


from sklearn.linear_model import LogisticRegression
lrmodel=LogisticRegression()
lrmodel.fit(X_train,y_train)
pred=lrmodel.predict_proba(X_test)

prob=[i for i in zip(*pred)][1]
final=pd.Series(data=prob,index=y_test.index)
final
'''

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [196]:
'''def new_connections_predictions():
    

    
    return final'''

In [201]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn_soundarajan_hopcroft['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    df = df.join(df_cn_soundarajan_hopcroft,how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df = future_connections.join(df,how='outer')
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba,X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    return target['prob']

In [202]:
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0, 9)          0.027476
(0, 19)         0.062514
(0, 20)         0.237355
(0, 35)         0.014795
(0, 38)         0.009423
(0, 42)         0.825917
(0, 43)         0.018845
(0, 50)         0.010957
(0, 53)         0.543638
(0, 54)         0.042739
(0, 62)         0.392491
(0, 63)         0.047272
(0, 69)         0.023942
(0, 72)         0.011464
(0, 83)         0.175174
(0, 90)         0.029057
(0, 91)         0.017145
(0, 95)         0.114310
(0, 100)        0.027313
(0, 106)        0.948429
(0, 108)        0.016087
(0, 109)        0.011638
(0, 110)        0.011292
(0, 118)        0.010161
(0, 122)        0.029446
(0, 131)        0.040706
(0, 133)        0.688777
(0, 140)        0.073574
(0, 149)        0.137029
(0, 151)        0.009471
                  ...   
(988, 989)      0.011923
(988, 996)      0.011931
(988, 997)      0.059098
(988, 1000)     0.011917
(988, 1002)     0.011931
(989, 994)      0.011928
(989, 996)      0.011928
(989, 1003)     0.011928
(989, 1004)     0.011928


In [206]:
final

(0, 9)          0.027858
(0, 19)         0.065261
(0, 20)         0.263959
(0, 35)         0.013283
(0, 38)         0.008472
(0, 42)         0.764182
(0, 43)         0.018364
(0, 50)         0.010359
(0, 53)         0.463335
(0, 54)         0.043002
(0, 62)         0.412424
(0, 63)         0.044442
(0, 69)         0.021732
(0, 72)         0.011002
(0, 83)         0.190807
(0, 90)         0.028255
(0, 91)         0.016034
(0, 95)         0.120269
(0, 100)        0.027319
(0, 106)        0.953958
(0, 108)        0.014792
(0, 109)        0.011225
(0, 110)        0.010784
(0, 118)        0.009369
(0, 122)        0.029215
(0, 131)        0.040641
(0, 133)        0.737887
(0, 140)        0.071484
(0, 149)        0.145598
(0, 151)        0.008529
                  ...   
(988, 989)      0.011593
(988, 996)      0.011604
(988, 997)      0.022606
(988, 1000)     0.011586
(988, 1002)     0.011604
(989, 994)      0.011600
(989, 996)      0.011600
(989, 1003)     0.011467
(989, 1004)     0.011579
